<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_md")

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [5]:
def tokenize(doc):
    doc_tok = []
    for tok in doc:
        if (tok.is_stop == False) and (tok.is_punct == False):
            doc_tok.append(tok.text.lower())
    return(doc_tok)

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [6]:
vect = TfidfVectorizer(stop_words='english', max_features=5000)
vect.fit(yelp['text'])
vectorized = vect.transform(yelp['text'])

In [7]:
vectorized.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')

nn.fit(vectorized.todense())

#test get k neighbors for first review
nn.kneighbors(vectorized.todense()[0])

(array([[0.        , 1.        , 1.        , 1.25325603, 1.25954602,
         1.26787687, 1.26906557, 1.28930992, 1.29809138, 1.30003428]]),
 array([[   0, 6311, 6204, 2131, 3276, 2943, 6019, 8470, 9036,   14]]))

In [9]:
fake_review_angry = 'terrible service, rude staff, will no be coming back!'
nn.kneighbors(vect.transform([fake_review_angry]).todense())

(array([[0.97929807, 1.        , 1.        , 1.06780361, 1.09064038,
         1.1389684 , 1.14119696, 1.15136631, 1.15372018, 1.16245423]]),
 array([[5298, 6311, 6204,  372, 2627, 9375, 2406, 7978, 7548, 6391]]))

In [10]:
# Looks like it worked!
yelp['text'][5298]

'Terrible.. absolutely terrible. Dr. Hinds is especially rude, very difficult to get anyone to return a phone call, just overall terrible patient care.'

In [11]:
# And another...
yelp['text'][372]

"If you're a fan of explosive diarrhea and really rude staff this is the place for you."

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [12]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from numpy.random import randint

In [13]:
vect = TfidfVectorizer(stop_words='english', max_features=5000)
gbc = SGDClassifier()

pipe = Pipeline([('vect', vect), ('clf', gbc)])

In [14]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'clf__max_iter':(20, 10, 100)
}

In [15]:
grid_search = GridSearchCV(pipe,parameters, cv=4, n_jobs=4, verbose=10)

In [16]:
grid_search.fit(yelp['text'], yelp['stars'])

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    4.7s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    6.6s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    9.8s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   12.0s
[Parallel(n_jobs=4)]: Done  33 out of  36 | elapsed:   16.4s remaining:    1.5s
[Parallel(n_jobs=4)]: Done  36 out of  36 | elapsed:   17.4s finished


GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=5000,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

In [17]:
best = grid_search.best_estimator_

In [18]:
predict_angry = best.predict([fake_review_angry])

In [19]:
# angry review predicts one star!
predict_angry

array([1])

In [20]:
predict_meh = best.predict(['The staff were polite, but there was a huge line at the checkout. meh'])

In [21]:
# Meh review predicts 2 stars
predict_meh

array([2])

In [22]:
predict_awesome = best.predict(['This place is a gem. Amazing food, friendly staff, and the one waiter remembers the appetizer we usually order and puts it in when we come in the door'])

In [23]:
# Well, my model seems to be working pretty well. 
predict_awesome

array([5])

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [24]:
import gensim
import numpy as np
import re
from gensim import corpora
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import smart_open, simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
import matplotlib.pyplot as plt

In [25]:
def gensim_tokenize(text):
    return(token for token in simple_preprocess(text) if token not in STOPWORDS)

In [26]:
def doc_streamer(series_docs):
    for f in series_docs:
        text = f.strip()
        tokens = gensim_tokenize(str(text))
        yield(tokens)

In [27]:
id2word = corpora.Dictionary(doc_streamer(yelp['text']))
id2word.filter_extremes(no_below=10, no_above=0.75)
corp = [id2word.doc2bow(t) for t in doc_streamer(yelp['text'])]

In [28]:
lda = LdaMulticore(corpus = corp,
                   id2word = id2word,
                   random_state = 42,
                   num_topics = 8,
                   passes = 10,
                   workers = 4
                  )

In [ ]:
lda

##Topics

The first topic seems to be good reviews about food. In particular pizza restaurants. My model lists high
relevence for "food", "good", "chicken", and "pizza". Pizza places tend to have wings. The second is general good reviews of restaurants, talking about good service. The third is good reviews about the speed of the service, again the most relevent word being food. 

In [56]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [57]:
pyLDAvis.gensim.prepare(lda, corp, id2word)

/home/joe2/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.149396 -0.007708       1        1  26.195028
5      0.112178 -0.007634       2        1  15.265973
6      0.007328  0.103914       3        1  13.068764
3     -0.114084  0.077308       4        1  12.235655
7     -0.090723  0.010653       5        1  11.528442
4     -0.151694 -0.062503       6        1   9.985993
0     -0.010637 -0.117134       7        1   5.938990
1      0.098235  0.003106       8        1   5.781157, topic_info=     Category         Freq        Term        Total  loglift  logprob
133   Default  5015.000000        food  5015.000000  30.0000  30.0000
667   Default   534.000000       cream   534.000000  29.0000  29.0000
188   Default  4304.000000       great  4304.000000  28.0000  28.0000
671   Default   437.000000         ice   437.000000  27.0000  27.0000
122   Default  4737.000000       place  4737.000000  26.0000  26.0000
94    Default   992.000000        room   992.000000  25.0000  25.0000
318   Default  1240.000000   delicious  1240.000000  24.0000  24.0000
37    Default  4611.000000        good  4611.000000  23.0000  23.0000
189   Default   400.000000        hair   400.000000  22.0000  22.0000
1667  Default   627.000000         car   627.000000  21.0000  21.0000
342   Default  1519.000000       order  1519.000000  20.0000  20.0000
22    Default   968.000000     minutes   968.000000  19.0000  19.0000
168   Default  1145.000000   recommend  1145.000000  18.0000  18.0000
1039  Default  1431.000000     chicken  1431.000000  17.0000  17.0000
309   Default   830.000000      cheese   830.000000  16.0000  16.0000
176   Default  1360.000000     amazing  1360.000000  15.0000  15.0000
992   Default   610.000000      coffee   610.000000  14.0000  14.0000
1223  Default   300.000000   chocolate   300.000000  13.0000  13.0000
638   Default  1619.000000        love  1619.000000  12.0000  12.0000
42    Default  3659.000000     service  3659.000000  11.0000  11.0000
702   Default   714.000000       store   714.000000  10.0000  10.0000
1029  Default  1598.000000     ordered  1598.000000   9.0000   9.0000
2024  Default   238.000000          dr   238.000000   8.0000   8.0000
1509  Default   454.000000       hotel   454.000000   7.0000   7.0000
31    Default  3470.000000        time  3470.000000   6.0000   6.0000
328   Default   941.000000       fresh   941.000000   5.0000   5.0000
473   Default   936.000000        work   936.000000   4.0000   4.0000
353   Default   832.000000       sauce   832.000000   3.0000   3.0000
374   Default   850.000000       asked   850.000000   2.0000   2.0000
881   Default  1152.000000         new  1152.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
1138   Topic8    70.192947      brunch   188.951996   1.8603  -5.8615
960    Topic8    39.711811       sugar    82.308884   2.1217  -6.4311
1257   Topic8    89.774277       rolls   273.633240   1.7361  -5.6154
1137   Topic8   121.679573   breakfast   457.338440   1.5265  -5.3113
309    Topic8   176.223648      cheese   830.460754   1.3003  -4.9409
2170   Topic8    29.843182      yogurt    56.674606   2.2092  -6.7167
1191   Topic8    44.294285      creamy   110.324203   1.9380  -6.3218
318    Topic8   171.465393   delicious  1240.435547   0.8717  -4.9683
682    Topic8   104.060188       sushi   541.091064   1.2019  -5.4677
122    Topic8   309.913055       place  4737.444824   0.1236  -4.3764
2051   Topic8    60.439892        thai   219.896561   1.5591  -6.0111
37     Topic8   285.037994        good  4611.111816   0.0670  -4.4601
44     Topic8   101.187660       sweet   617.156921   1.0424  -5.4957
68     Topic8   178.388565         got  2085.104980   0.3920  -4.9287
937    Topic8   103.759491    location   882.862549   0.7095  -5.4706
104    Topic8   141.771072          ve  1946.053833   0.2312  -5.1585
342    Topic8   127.306175       order  1519.779663   0.3

In [63]:
from gensim.models.coherencemodel import CoherenceModel


In [64]:
def get_coherence_vals(dictionary, corp, series_in, limit, start=2, step=3):
    coherence_vals = []
    model_lst = []
    for n_topics in range(start, limit, step):
        stream = doc_streamer(series_in)
        model = LdaMulticore(corpus=corp, num_topics=n_topics, id2word=id2word, workers=4)
        model_lst.append(model)
        coherencemodel = CoherenceModel(model=model, texts=stream, dictionary=dictionary, coherence='c_v')
        coherence_vals.append(coherencemodel.get_coherence())
    return(model_lst, coherence_vals)

In [ ]:
model_list, coherence_values = get_coherence_vals(dictionary=id2word, 
                                                        corp=corp, 
                                                        series_in=yelp['text'], 
                                                        start=2, 
                                                        limit=40, 
                                                        step=6)

/home/joe2/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/joe2/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/joe2/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


In [ ]:
# Graph
limit = 40;
start = 2;
step = 6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlamel('Number of topics')
plt.ylabel('coherence score')
plt.show()

In [ ]:
for m, cv in zip(x, coherence_values):
    print('Number of Topics: ', m, " Coherence value for {} topics: ".format(m), cv)

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)